# Установка и настройка окружения

In [1]:
!pip install langchain langchain_gigachat langchain_community langchain_openai pandas python_dotenv langgraph -q


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [22]:
# pip install python-dotenv
from dotenv import find_dotenv, load_dotenv
load_dotenv(find_dotenv())

True

# Загружаем исходный каталог товаров

In [3]:
import pandas as pd

# Load the CSV file into a DataFrame with ';' as the separator
stuff_catalogue_df = pd.read_csv('stuff_catalogue.csv', sep=';')
print(stuff_catalogue_df.head())

   №                                Наименование товара  Цена руб  \
0  1  Молоко восстановленное из сухого молока пастер...    105.00   
1  2  Молоко восстановленное из сухого молока ультра...     99.10   
2  3  Молоко восстановленное из сухого молока ультра...     92.35   
3  4  Молоко восстановленное из сухого молока пастер...     56.75   
4  5  Молоко питьевое ультрапастеризованное топленое...    111.55   

  Торговая марка Режим хранения  Срок реализации. мес        Упаковка  \
0      Лазовская           2 +4                    10  пласт. Бутылка   
1      Лазовское          2 +25                   180             ТФА   
2      Лазовское          2 +25                   180             ТФА   
3      Лазовское           2 +4                    10         Пюр-пак   
4      Лазовское         2..+25                   180             ТФА   

                                            Описание  
0    Пастеризованное молоко. Массовая доля жира 3.2%  
1  Ультрапастеризованное молоко — мо

# Инициализируем подключение к GigaChat

In [10]:
from typing import List, Literal

from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from langchain_gigachat import GigaChat
from pydantic import BaseModel, Field
from tenacity import retry, stop_after_attempt

In [26]:
giga = GigaChat(model="GigaChat-Max", profanity_check=False, verify_ssl_certs=False)
giga.invoke("Привет!").content

'Здравствуйте! Рад вас видеть. Как я могу вам помочь?'

# Создаем цепочку для извлечения структурированных данных

In [27]:
# Описание структуры данных, которую будем извлекать из каждой строчки csv
from stuff_utils import StuffDesc

Здесь эксперты должны сообщить модели свои экспертные знания

In [28]:
STUFF_EXTRACTION_PROMPT = """Ты - агент-товаровед, который занимается систематизацией базы данных товаров.
Ты должен изучить описание товары и извлечь из него информацию согласно требуемой схеме в JSON.
Если каких-то данных не хватает, пиши n/a для строк и -1 для чисел.

ТФА, ТБА, фин-пак, дой-пак и т.д. это пакет. ж/б - железная банка. п/б - пластиковая бутылка. ст/б - стеклянная бутылка
Если данных о минимальной и максимальной температуре нет, а она задана явно - пиши одинаковое число для минимума и максимума. Тоже с жирностью.
Если тип упаковки не попадает ни в одной из категорий - выбирай "другое"
Если поле NaN или не указана, пиши n/a для текстовых полей и -1 для числовых.

Вот описание товара, которое тебе дано:
<STUFF_INFO>
{stuff_info}
</STUFF_INFO>

Выведи только следующую информацию в формате JSON. Не окружай ответ тегами ```json ...```:
{format_instructions}
"""

@retry(stop=stop_after_attempt(3))
def extract_info(stuff_info: str) -> StuffDesc:
    parser = PydanticOutputParser(pydantic_object=StuffDesc)
    prompt = ChatPromptTemplate.from_messages([
        ("system", STUFF_EXTRACTION_PROMPT)
    ]).partial(format_instructions=parser.get_format_instructions())
    
    chain = prompt | giga | parser

    return chain.invoke(
        {
            "stuff_info": stuff_info,
        }
    )


# Извлекаем структурированый объект из каждой строки таблицы

In [29]:
stuff_desc_list = []

for index, row in stuff_catalogue_df.iterrows():
    try:
        # print(str(row))
        stuff_desc = extract_info(str(row))
        stuff_desc_list.append(stuff_desc)
        print(stuff_desc)
    except Exception as e:
        print("Error:", e)
        print("Can't parse row: ", row)

stuff_id='1' desc='Пастеризованное молоко. Массовая доля жира 3.2%' brand='Лазовская' price=105.0 min_store_temperature=2 max_store_temperature=4 shelf_life=10 type_of_packing='пластиковая бутылка' mass_fraction_of_fat_min=3.2 mass_fraction_of_fat_max=3.2
stuff_id='2' desc='Ультрапастеризованное молоко — молоко длительного хранения.' brand='Лазовское' price=99.1 min_store_temperature=2 max_store_temperature=25 shelf_life=180 type_of_packing='пакет' mass_fraction_of_fat_min=-1.0 mass_fraction_of_fat_max=-1.0
stuff_id='3' desc='Ультрапастеризованное молоко — молоко длительного хранения.' brand='Лазовское' price=92.35 min_store_temperature=2 max_store_temperature=25 shelf_life=180 type_of_packing='пакет' mass_fraction_of_fat_min=-1.0 mass_fraction_of_fat_max=-1.0
stuff_id='4' desc='Пастеризованное молоко.' brand='Лазовское' price=56.75 min_store_temperature=2 max_store_temperature=4 shelf_life=10 type_of_packing='пакет' mass_fraction_of_fat_min=3.2 mass_fraction_of_fat_max=3.2
stuff_id='5

# Сохраняем полученные результаты в файл

In [30]:
len(stuff_desc_list)

48

In [31]:
# Save stuff_desc_list to JSON
import json

with open('stuff_desc_list.json', 'w') as f:
    json.dump([stuff_desc.dict() for stuff_desc in stuff_desc_list], f, indent=4, ensure_ascii=False)

/var/folders/1m/7jmnmlbs2hsbw3qkcx46f0l86hy6kp/T/ipykernel_57761/1720473458.py:5: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  json.dump([stuff_desc.dict() for stuff_desc in stuff_desc_list], f, indent=4, ensure_ascii=False)
